# Analysis of AIDA impact simulations

## 0) Imports

In [16]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import h5py
import scipy
from scipy import constants

%load_ext nb_black
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option("display.precision", 2)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## 1) Load simulation data:

In [6]:
dirpath = "../data/test_viscosity_9/visc1/"
filename = "impact_damage_visc1.0089.h5"

<IPython.core.display.Javascript object>

In [7]:
with h5py.File(f"{dirpath}{filename}", "r") as hdf:
    keys = list(hdf.keys())

<IPython.core.display.Javascript object>

In [8]:
keys

['DIM_root_of_damage_porjutzi',
 'DIM_root_of_damage_tensile',
 'a',
 'a_grav',
 'activation_thresholds',
 'alpha_jutzi',
 'dalphadt',
 'damage_total',
 'ddamage_porjutzidt',
 'dddt',
 'ddeviatoric_stress_dt',
 'dedt',
 'deviatoric_stress',
 'drhodt',
 'e',
 'local_strain',
 'm',
 'material_type',
 'maximum_number_of_flaws',
 'number_of_activated_flaws',
 'number_of_interactions',
 'p',
 'rho',
 'sml',
 'soundspeed',
 'time',
 'tree_depth',
 'v',
 'x']

<IPython.core.display.Javascript object>

In [64]:
with h5py.File(f"{dirpath}{filename}", "r") as hdf:
    positions = hdf.get("x")[:]
    velocities = hdf.get("v")[:]
    mass = hdf.get("m")[:]

<IPython.core.display.Javascript object>

In [65]:
df = pd.DataFrame(
    {
        "x": positions[:, 0],
        "y": positions[:, 1],
        "z": positions[:, 2],
        "v_x": velocities[:, 0],
        "v_y": velocities[:, 1],
        "v_z": velocities[:, 2],
        "m": mass,
    }
)

<IPython.core.display.Javascript object>

In [66]:
df

,x,y,z,v_x,v_y,v_z,m
0,-0.48,-0.30,-9.75,-0.02,-1.61e-02,-0.55,51.75
1,0.13,-0.30,-9.75,0.01,-1.60e-02,-0.55,51.75
2,-0.48,0.07,-9.75,-0.02,6.37e-03,-0.55,51.75
3,0.13,0.07,-9.75,0.01,6.14e-03,-0.55,51.75
4,-0.48,0.44,-9.75,-0.02,2.70e-02,-0.54,51.75
...,...,...,...,...,...,...,...
99740,3.41,-1.00,5.16,471.58,-1.29e+02,798.59,8.93
99741,0.20,0.71,-1.08,10.65,-1.24e+02,8.84,8.93
99742,3.36,1.31,5.29,465.09,1.90e+02,819.28,8.93
99743,-0.22,3.70,7.12,30.25,4.91e+02,1119.42,8.93


<IPython.core.display.Javascript object>

## 2) Beta factor

###  2.0) Physical parameters

In [302]:
rho_didymoon = 2860.0
r_surface = 80
M = 4 / 3 * np.pi * r_surface ** 3 * rho_didymoon
G = scipy.constants.G

impactor_momentum = 500 * 6000 ** 2

<IPython.core.display.Javascript object>

### 2.1) Escape velocity

In [303]:
df["v_escape"] = np.sqrt(2 * G * M / (r_surface + df.z))

<IPython.core.display.Javascript object>

### 2.2) Conditions for particles to influence beta factor

In [328]:
filt_beta = df["v_z"] > 0  # & (np.abs(df["v_z"]) > df["v_escape"])  # & (df["z"] > 0)

<IPython.core.display.Javascript object>

In [329]:
df_beta = df[filt_beta]

<IPython.core.display.Javascript object>

### 2.3) Compute beta factor

In [330]:
recoil_momentum = (df[filt_beta]["m"] * df[filt_beta]["v_z"] ** 2).sum()

<IPython.core.display.Javascript object>

In [331]:
beta = recoil_momentum / impactor_momentum
beta

0.10460891651163402

<IPython.core.display.Javascript object>

In [332]:
filt_beta.sum()

32736

<IPython.core.display.Javascript object>

In [333]:
df[filt_beta].sort_values("v_z", ascending=False).head(50)

,x,y,z,v_x,v_y,v_z,m,v_escape
99690,-0.48,-8.63e+00,20.95,-60.51,-1113.49,2968.19,8.93,0.09
99696,-0.48,8.88e+00,20.77,-60.68,1156.00,2938.64,8.93,0.09
99693,-8.44,1.82e+00,20.69,-1079.21,241.12,2936.76,8.93,0.09
99691,-8.51,-1.91e+00,20.63,-1088.77,-243.36,2928.82,8.93,0.09
99698,6.67,-3.09e+00,20.32,880.99,-373.28,2898.02,8.93,0.09
99702,6.79,3.09e+00,20.22,896.62,389.23,2881.64,8.93,0.09
99700,7.44,-3.98e-02,19.95,986.22,0.16,2846.40,8.93,0.09
99689,-5.80,-6.37e+00,20.03,-707.78,-802.89,2823.82,8.93,0.09
99694,0.89,1.29e+00,19.53,138.49,156.56,2813.59,8.93,0.09
99695,-5.56,6.42e+00,19.94,-674.11,818.62,2809.07,8.93,0.09


<IPython.core.display.Javascript object>